In [1]:
#export
import k1lib, time, torch; from k1lib import fmt
from k1lib.callbacks import Callback, Cbs

In [2]:
#export
_ltime = 15; _lt1 = 8; _lt2 = 18
class TimeData:
    def __init__(self, tProfiler, mS:k1lib.selector.ModuleSelector):
        self.tProfiler = tProfiler; self.mS = mS
        self.startTime = None; self.time = 0; self.hook()
    def hook(self):
        def fpHk(m, i):
            if self.tProfiler.is_cuda: torch.cuda.synchronize()
            self.startTime = time.time()
        def fHk(m, i, o):
            if self.tProfiler.is_cuda: torch.cuda.synchronize()
            self.time += time.time() - self.startTime
        self.fpH = self.mS.nn.register_forward_pre_hook(fpHk)
        self.fH = self.mS.nn.register_forward_hook(fHk)
    def unhook(self):
        self.tProfiler.totalTime = max(self.tProfiler.totalTime, self.time)
        self.fpH.remove(); self.fH.remove()
    def __getstate__(self):
        answer = dict(self.__dict__)
        del answer["mS"]; del answer["tProfiler"]; return answer
    def __setstate__(self, state): self.__dict__.update(dict(state))
    def __str__(self):
        if self.time <= 1e-20: return ""
        a = f"{fmt.time(self.time)}".ljust(_ltime)
        b = f"{round(100 * self.time / self.tProfiler.totalTime)}%".rjust(_lt1)
        c = f"{round(100 * self.time / self.tProfiler.selectedMaxTime)}%".rjust(_lt2) if self.tProfiler.selectedMaxTime != None and "_timeProf_" in self.mS else ""
        return f"{a}{b}{c}"

In [3]:
#export
class TimeProfiler(Callback):
    """Profiles execution time. Only measures forward times, as
backward times can't really be measured"""
    def startRun(self):
        if not hasattr(self, "selector"): # if no selectors found
            self.selector = self.l.model.select("")
        for m in self.selector.modules(): m.data = TimeData(self, m)
        self.selector.displayF = lambda m: (fmt.txt.red if "_timeProf_" in m else fmt.txt.identity)(m.data)
        self.totalTime = 0; self.selectedMaxTime = None
    def startStep(self): return True
    def run(self):
        """Runs everything"""
        with self.cbs.context(), self.cbs.suspendEval():
            self.is_cuda = next(self.l.model.parameters()).is_cuda
            if self.is_cuda: self.cbs.add(Cbs.Cuda())
            else: self.cbs.add(Cbs.Cpu())
            self.l.run(1, 1)
        for m in self.selector.modules(): m.data.unhook()
    def css(self, css:str):
        """Selects a small part of the network to highlight"""
        self.selector.parse(k1lib.selector.preprocess(css, "_timeProf_"))
        self.selectedMaxTime = 0
        for m in self.selector.modules():
            if "_timeProf_" in m:
                self.selectedMaxTime = max(self.selectedMaxTime, m.data.time)
        print(self.__repr__())
        self.selector.clearProps(); self.selectedMaxTime = None
    def __repr__(self):
        header = "time".ljust(_ltime) + "% total".rjust(_lt1) + ("% selected max" if self.selectedMaxTime != None else "").rjust(_lt2)
        footer = ""
        if self.selectedMaxTime != None:
            b = f"{round(100 * self.selectedMaxTime / self.totalTime)}%".rjust(_lt1, " ")
            st = f"{fmt.time(self.selectedMaxTime)}".rjust(_lt2)
            footer = ("Selected max", " " * _ltime + b + st)
        return f"""TimeProfiler ({"GPU" if self.is_cuda else "CPU"}):
{k1lib.tab(self.selector.__repr__(intro=False, header=header, footer=footer))}

Caveats: This one's a bit stranger than memory and computation profilers
1. There is no "total" time (adding all times in all modules). There
    is total network execution time tho, which is just the time taken
    for the top level module to execute
2. "% selected max" column displays percentage of selected max, not
    percentage of total selected time, which may matter in your analysis

Can...
- tp.css("..."): highlights a particular part of the network
- tp.selector: to get internal k1lib.ModuleSelector object"""

In [4]:
l = k1lib.Learner.sample()
l.cbs.add(Cbs.Profiler())
l.Profiler.time.css("#lin1")

Epoch cancelled: Batch 1 reached.
TimeProfiler (CPU):
                                time            % total    % selected max
    root: Model                 520.71 us          100%
        lin1: LinBlock          228.17 us           44%              100%
            lin: Linear         113.25 us           22%    
            relu: ReLU          32.9 us              6%    
        lin2: Linear            28.85 us             6%    
    Selected max                                    44%         228.17 us

Caveats: This one's a bit stranger than memory and computation profilers
1. There is no "total" time (adding all times in all modules). There
    is total network execution time tho, which is just the time taken
    for the top level module to execute
2. "% selected max" column displays percentage of selected max, not
    percentage of total selected time, which may matter in your analysis

Can...
- tp.css("..."): highlights a particular part of the network
- tp.selector: to get int

In [5]:
!../../../export.py callbacks/profilers/time

Current dir: /home/kelvin/repos/labs/k1lib, ../../../export.py
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 0.8
Uninstalling k1lib-0.8:
  Successfully uninstalled k1lib-0.8
running install
running bdist_egg
running egg_info
creating k1lib.egg-info
writing k1lib.egg-info/PKG-INFO
writing dependency_links to k1lib.egg-info/dependency_links.txt
writing requirements to k1lib.egg-info/requires.txt
writing top-level names to k1lib.egg-info/top_level.txt
writing manifest file 'k1lib.egg-info/SOURCES.txt'
reading manifest file 'k1lib.egg-info/SOURCES.txt'
adding license file 'LICENSE'
writing manifest file 'k1lib.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/k1lib
copying k1lib/_learner.py -> build/lib/k1lib
copying k1lib/fmt.py -> build/lib/k1lib
copying k1lib/selector.py -> build/lib/k1lib
copying k1lib/imports.py -> build/